In [4]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn import  datasets
from sklearn.model_selection import  train_test_split

cancer = datasets.load_breast_cancer()
X = cancer.data
y = cancer.target

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=1/5,random_state=8)


X_train


xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_test = xgb.DMatrix(X_test, label=y_test)


params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eta": "0.1",
    "min_child_weight": 1,
    "max_depth": 5,
}


num_round = 50
watchlist = [(xgb_train, 'train'), (xgb_test, 'test')]

bst = xgb.train(params, xgb_train, num_round, watchlist)

xgb_test.save_binary('dtest.buffer')
xgb_test2 =  xgb.DMatrix('dtest.buffer')

preds2 = bst.predict(xgb_test2)

print(preds2)



[21:12:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.60907	test-logloss:0.61263
[1]	train-logloss:0.53994	test-logloss:0.54883
[2]	train-logloss:0.48091	test-logloss:0.49563
[3]	train-logloss:0.43008	test-logloss:0.44823
[4]	train-logloss:0.38704	test-logloss:0.41053
[5]	train-logloss:0.35003	test-logloss:0.37543
[6]	train-logloss:0.31740	test-logloss:0.34200
[7]	train-logloss:0.28815	test-logloss:0.31464
[8]	train-logloss:0.26270	test-logloss:0.28995
[9]	train-logloss:0.24026	test-logloss:0.26890
[10]	train-logloss:0.21968	test-logloss:0.24842
[11]	train-logloss:0.20116	test-logloss:0.23137
[12]	train-logloss:0.18456	test-logloss:0.21769
[13]	train-logloss:0.16956	test-logloss:0.20432
[14]	train-logloss:0.15608	test-logloss:0.19283
[15]	train-logloss:0.14405	test-loglos

# 多种格式数据加载

In [10]:

import scipy

row = []; col = [] ; dat =[]
i =0
for arr in X_train:
    for k,v in enumerate(arr):
        row.append(i);col.append(int(k));dat.append(float(v))
    i+=1

csr = scipy.sparse.csr_matrix((dat,(row,col)))
csr

xgb_train = xgb.DMatrix(csr,label=y_train)
watchlist = [(xgb_test, 'eval'), (xgb_train, 'train')]

bst = xgb.train(params,xgb_train,num_round,watchlist)

nparr = csr.todense()

xgb_train = xgb.DMatrix(nparr,label=y_train)
watchlist = [(xgb_test, 'eval'), (xgb_train, 'train')]

bst = xgb.train(params,xgb_train,num_round,watchlist)

[21:14:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.61263	train-logloss:0.60907
[1]	eval-logloss:0.54883	train-logloss:0.53994
[2]	eval-logloss:0.49563	train-logloss:0.48091
[3]	eval-logloss:0.44823	train-logloss:0.43008
[4]	eval-logloss:0.41053	train-logloss:0.38704
[5]	eval-logloss:0.37543	train-logloss:0.35003
[6]	eval-logloss:0.34200	train-logloss:0.31740
[7]	eval-logloss:0.31464	train-logloss:0.28815
[8]	eval-logloss:0.28995	train-logloss:0.26270
[9]	eval-logloss:0.26890	train-logloss:0.24026
[10]	eval-logloss:0.24842	train-logloss:0.21968
[11]	eval-logloss:0.23137	train-logloss:0.20116
[12]	eval-logloss:0.21769	train-logloss:0.18456
[13]	eval-logloss:0.20432	train-logloss:0.16956
[14]	eval-logloss:0.19283	train-logloss:0.15608
[15]	eval-logloss:0.18216	train-loglos

In [ ]:
pred_train = bst.predict(xgb_train, output_margin=True)
pred_test = bst.predict(xgb_test, output_margin=True)


xgb_train.set_base_margin(pred_train)
xgb_test.set_base_margin(pred_test)

print("以下是基于初始预测值后进行运行结果")
bst = xgb.train(params,xgb_train,num_round,watchlist)

